In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#for SVM and XGBOOST
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score,precision_recall_curve, auc


import xgboost as xgb

In [10]:
df = pd.read_csv("df_final")

In [3]:
print(df.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'game', 'round', 'start round', 'hit',
       'duration round', 'participant', 'accelarator press',
       'accelarator first signal', 'start heartrate in video',
       'start heartrate in actual time', 'hit, correct', 'hit, incorrect',
       'other hit, correct', 'other hit, incorrect', 'confidence score',
       'experience score', 'games played prior on current day', 'start video',
       'favorite fruit_prume', 'favorite fruit_strawberry',
       'accelarator device_band', 'accelarator device_elastics',
       'accelarator press_seconds', 'accelarator first signal_seconds',
       'hit_seconds', 'start heartrate in video_seconds',
       'duration round_seconds', 'duration_next_round', 'no_hit',
       'duration_round_seconds', 'accelerator_press_hit_adj_time',
       'accelerator_signal_hit_adj_time', 'heartrate_adj_time', 'before3_mean',
       'before3_min', 'before3_max', 'before2_mean', 'before2_min',
       'before2_max', 'before1_mean',

In [12]:
df['target'] = df[['hit, correct', 'hit, incorrect','other hit, correct', 'other hit, incorrect', 'no_hit']].idxmax(axis=1)
# 0: hit, correct 1: hit,incorrect 2:other hit, correct 3: other hit, incorrect 4: no hit

df = df.drop(columns=['hit, correct', 'hit, incorrect','other hit, correct', 'other hit, incorrect', 'no_hit'])

# Encode the target variable as categorz
df['target'] = df['target'].astype('category').cat.codes

In [13]:
# drop nan lines
#print(df.isna().sum())
df = df.dropna()
df.reset_index(drop=True)



,Unnamed: 0.1,Unnamed: 0,game,round,start round,hit,duration round,participant,accelarator press,accelarator first signal,...,after2_mean,after2_min,after2_max,after3_mean,after3_min,after3_max,max_acc_value,max_acc_time,time_diff_max_acc,target
0,0,46,1,1,00:02:21:690,00:02:29:840,00:00:08:150,Paula,00:00:05:660,00:00:06:120,...,92.19320,90.6071,93.7793,99.11715,97.5293,100.7050,6.091317,143.527913,-0.192087,0
1,1,38,1,1,00:02:21:690,00:02:29:840,00:00:08:150,Paula,00:00:05:660,00:00:06:120,...,65.43120,65.2600,65.6024,67.92575,66.7111,69.1404,0.267501,178.799603,-0.080397,2
2,2,47,1,2,00:03:16:840,00:03:40:240,00:00:23:300,Paula,00:00:05:660,00:00:06:120,...,69.62235,69.4123,69.8324,70.38970,70.2741,70.5550,5.137283,214.021673,-0.098327,3
3,3,48,1,3,00:05:57:630,00:05:58:840,00:00:01:210,Paula,00:00:05:660,00:00:06:120,...,72.88755,72.0288,73.7463,69.44310,68.7600,70.1262,3.195546,352.246442,-0.473558,3
4,4,49,1,4,00:06:55:390,00:06:59:780,00:00:04:390,Paula,00:00:05:660,00:00:06:120,...,96.31210,95.6328,97.1188,97.28170,96.5562,98.0072,4.372432,413.656202,-0.003799,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,276,277,5,36,00:14:10:030,00:14:13:450,00:00:03:420,Luna,00:00:41:210,00:00:41:360,...,75.01560,74.3310,75.7002,75.40630,75.1315,75.6811,6.900037,812.080407,-0.009593,0
235,277,33,5,36,00:14:10:030,00:14:13:450,00:00:03:420,Luna,00:00:41:210,00:00:41:360,...,84.04635,83.0105,85.0822,83.32325,82.2820,84.3645,1.185744,821.890437,-1.749563,2
236,278,278,5,37,00:14:19:240,00:14:38:240,00:00:19:000,Luna,00:00:41:210,00:00:41:360,...,73.53885,73.3676,73.7101,73.21540,73.1886,73.2422,24.840476,836.875273,-0.004727,1
237,279,279,5,38,00:14:44:390,00:14:53:300,00:00:08:810,Luna,00:00:41:210,00:00:41:360,...,80.94185,80.4505,81.4332,79.92755,79.5123,80.3428,5.753286,851.930117,-0.009883,1


In [17]:
print(len(df))

239


In [15]:
#features

X = df[['confidence score','before1_mean', 'before1_min', 'before1_max',
       'after1_mean', 'after1_min', 'after1_max','max_acc_value', 'time_diff_max_acc']]
y = df['target']


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [27]:
#Multiclass SVM
svm_model = SVC(class_weight='balanced', decision_function_shape='ovr')

# Train the model
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced')

In [28]:

# Make predictions
y_pred = svm_model.predict(X_test)

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
print(confusion_matrix(y_test, y_pred))

# If you want to calculate ROC-AUC for multiclass
y_test_bin = pd.get_dummies(y_test)
y_pred_bin = pd.get_dummies(y_pred)
roc_auc = roc_auc_score(y_test_bin, y_pred_bin, average='macro')
print(f'ROC-AUC Score: {roc_auc}')

              precision    recall  f1-score   support

           0       0.53      0.50      0.51        20
           1       0.14      0.50      0.22         2
           2       0.21      0.43      0.29         7
           3       1.00      0.06      0.12        16
           4       0.00      0.00      0.00         3

    accuracy                           0.31        48
   macro avg       0.38      0.30      0.23        48
weighted avg       0.59      0.31      0.30        48

[[10  2  7  0  1]
 [ 0  1  1  0  0]
 [ 0  1  3  0  3]
 [ 7  3  2  1  3]
 [ 2  0  1  0  0]]
ROC-AUC Score: 0.5615359836051776


In [25]:
# XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Predict probabilities
y_probs = xgb_model.predict_proba(X_test)[:, 1]

# Calculate precision-recall curve for positive class 'hit_correct' (class 0 in binary)
precision, recall, thresholds = precision_recall_curve(y_test == 0, y_probs)

# Calculate AUC for precision-recall curve
auc_score = auc(recall, precision)

# Print results
print("Precision-Recall Curve:")
for p, r, t in zip(precision, recall, thresholds):
    print(f"Threshold: {t:.2f}, Precision: {p:.2f}, Recall: {r:.2f}")

# Evaluate the model
y_pred = xgb_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Precision-Recall Curve:
Threshold: 0.00, Precision: 0.42, Recall: 1.00
Threshold: 0.00, Precision: 0.43, Recall: 1.00
Threshold: 0.00, Precision: 0.41, Recall: 0.95
Threshold: 0.00, Precision: 0.42, Recall: 0.95
Threshold: 0.00, Precision: 0.41, Recall: 0.90
Threshold: 0.00, Precision: 0.40, Recall: 0.85
Threshold: 0.00, Precision: 0.38, Recall: 0.80
Threshold: 0.00, Precision: 0.37, Recall: 0.75
Threshold: 0.00, Precision: 0.38, Recall: 0.75
Threshold: 0.00, Precision: 0.38, Recall: 0.75
Threshold: 0.00, Precision: 0.39, Recall: 0.75
Threshold: 0.00, Precision: 0.41, Recall: 0.75
Threshold: 0.00, Precision: 0.42, Recall: 0.75
Threshold: 0.00, Precision: 0.43, Recall: 0.75
Threshold: 0.00, Precision: 0.44, Recall: 0.75
Threshold: 0.00, Precision: 0.45, Recall: 0.75
Threshold: 0.00, Precision: 0.47, Recall: 0.75
Threshold: 0.00, Precision: 0.45, Recall: 0.70
Threshold: 0.00, Precision: 0.47, Recall: 0.70
Threshold: 0.00, Precision: 0.45, Recall: 0.65
Threshold: 0.00, Precision: 0.46, Re